메모

In [1]:
map (*2) [1,2,3,4,5]

[2,4,6,8,10]

In [2]:
map :: (a -> b) -> [a] -> [b]
map f []     = []
map f (x:xs) = f x : map f xs

<span class='hoogle-name'><a target='_blank' href='http://hackage.haskell.org/packages/archive/base/latest/doc/html/Prelude.html#v:concat'>concat</a> &#x2237; [[a]] &#x2192; [a]</span>
<span class='hoogle-sub'>(<span class='hoogle-head'>package</span>
<span class='hoogle-package'>base</span>, <span class='hoogle-head'>module</span> <span class='hoogle-module'>Prelude</span>)</span>
<div class='hoogle-doc'>
<div class='hoogle-text'>Concatenate a list of lists. e.g.,</div>

```haskell
concat [xs1, xs2, xs3] == xs1 ++ xs2 ++ xs3
concat xss == foldr (++) [] xss
```

</div>

In [3]:
concat [[1,2], [3,4,5], [6,7,8,9]]

[1,2,3,4,5,6,7,8,9]

In [4]:
foldr (++) [] [[1,2], [3,4,5], [6,7,8,9]]

Line 1: Use concat
Found:
foldr (++) []
Why not:
concat

[1,2,3,4,5,6,7,8,9]

In [5]:
:doc foldr

In [6]:
:doc map

In [43]:
-- 변수 이름 타입은 문자열로 한다
type Nm = String

-- 람다식 문법 구조
data Tm = Var Nm | Lam Nm Tm | App Tm Tm  deriving Show

-- 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x t) = "\\" ++ x ++ " -> " ++ ppTm t
ppTm (App t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam{}) = paren (ppTm t)
  ppt t         = ppTm t
  pps s@(Var{}) = ppTm s
  pps s         = paren (ppTm s)

paren s = "(" ++ s ++ ")"

In [49]:
idTm = Lam "x" (Var "x")
ttTm = Lam "x" (Lam "y" (Var "x")) 
ffTm = Lam "x" (Lam "y" (Var "y")) 

putStr . ppTm $ Var "x"
putStr . ppTm $ idTm
putStr . ppTm $ App idTm idTm
putStr . ppTm $ App (App ttTm (Var "u")) (Var "w")

x

\x -> x

(\x -> x) (\x -> x)

(\x -> \y -> x) u w

In [50]:
-- 하스켈에서 그대로 붙여넣으면 실행도 어느 정도 가능하게
let z = "z" in (\x -> x) ((\x -> x) z)

Line 1: Use id
Found:
\ x -> x
Why not:
idLine 1: Use id
Found:
\ x -> x
Why not:
id

"z"

In [84]:
-- Tm의 구조에서 딱 한군데를 지정해 가리키기 위한 구조
data Tm' = Here Tm -- 여기야 여기
         | Var' Nm | Lam' Nm Tm' | App' Tm' Tm'
  deriving Show

-- 보기좋게 문자열로 변환해주는 함수
ppTm' (Here t) = "[" ++ ppTm t ++ "]"
ppTm' (Var' x) = x
ppTm' (Lam' x t) = "\\" ++ x ++ " -> " ++ ppTm' t
ppTm' (App' t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam'{})      = paren (ppTm' t)
  ppt t@(Here(Lam{})) = paren (ppTm' t)
  ppt t               = ppTm' t
  pps s@(Var'{})      = ppTm' s
  pps s@(Here(Var{})) = ppTm' s
  pps s               = paren (ppTm' s)

-- Tm을 Tm'으로 변환
tm2tm' (Var x) = Var' x
tm2tm' (Lam x t) = Lam' x (tm2tm' t)
tm2tm' (App t s) = App' (tm2tm' t) (tm2tm' s)

-- Tm'을 으로 변환 (Here 정보는 사라진다)
tm'2tm (Here t) = t
tm'2tm (Var' x) = Var x
tm'2tm (Lam' x t) = Lam x (tm'2tm t)
tm'2tm (App' t s) = App (tm'2tm t) (tm'2tm s) 

In [80]:
idTm' = tm2tm' idTm

Here(App idTm idTm)
putStr . ppTm' $ Here(App idTm idTm)

App' idTm' (Here(App idTm idTm))
putStr . ppTm' $ App' idTm' (Here(App idTm idTm))

Here (App (Lam "x" (Var "x")) (Lam "x" (Var "x")))

[(\x -> x) (\x -> x)]

App' (Lam' "x" (Var' "x")) (Here (App (Lam "x" (Var "x")) (Lam "x" (Var "x"))))

(\x -> x) ([(\x -> x) (\x -> x)])

In [81]:
-- 아래 람다식의 beta redex를 모두 찾아 각각의 경우를 하나씩 Here로 표시해 보자
putStr . ppTm $ Lam "y" (App idTm (App idTm (Var "y")))

\y -> (\x -> x) ((\x -> x) y)

In [82]:
-- 다음의 두 가지 경우가 있다
putStr . ppTm' $ Lam' "y" (Here(App idTm (App idTm (Var "y"))))
putStr . ppTm' $ Lam' "y" (App' idTm' (Here(App idTm (Var "y"))))

\y -> [(\x -> x) ((\x -> x) y)]

\y -> (\x -> x) ([(\x -> x) y])

In [78]:
-- 아래 람다식에는 beta redex가 3개 있는데 모두 찾아 각각을 Here로 표시해 보라
putStr . ppTm $ App idTm (App idTm idTm `App` App idTm idTm)

(\x -> x) ((\x -> x) (\x -> x) ((\x -> x) (\x -> x)))

In [ ]:
-- 여기에 해보세요

In [85]:
-- beta redex를 모두 찾아 리스트로 나열하는 함수를 작성하라
-- beta redex가 없을 경우는 빈 리스트가 결과값이 된다
redexes :: Tm -> [Tm']
redexes (Var x) = []
redexes (Lam x t) = undefined
redexes (App t s) = undefined